In [8]:
import pandas as pd
from dotenv import load_dotenv
import os
import pymysql

In [9]:
def create_pymysql_connect():
    """
    自動透過pymysql建立連線，回傳conn連線物件。
    所需各項資料請寫入.env檔案中。請勿直接寫於程式中。
    """

    load_dotenv()

    username = os.getenv("MYSQL_USERNAME")
    password = os.getenv("MYSQL_PASSWORD")
    target_ip = os.getenv("MYSQL_IP")
    target_port = int(os.getenv("MYSQL_PORTT"))
    db_name = os.getenv("MYSQL_DB_NAME")

    conn = pymysql.connect(
        host=target_ip,
        port=target_port,
        user=username,
        password=password,
        database=db_name,
        charset='utf8mb4'
    )

    return conn


def E_load_from_sql(table_name: str) -> pd.DataFrame:
    """
    輸入欲查詢的表名table_name，透過pymysql連線資料庫，
    並取得該表後將其轉成dataframe。

    連線所需資訊請寫入.env中，請勿寫入程式中。
    """

    conn = create_pymysql_connect()
    sql = f"SELECT * FROM {table_name}"

    try:
        df = pd.read_sql(sql, conn)
        return df.to_dict(orient='records')

    except Exception as e:
        raise Exception(f"讀取{table_name}表時發生錯誤：{e}")

In [ ]:
data = E_load_from_sql(table_name="salon")
df = pd.DataFrame(data=data)

df

C:\Users\add41\AppData\Local\Temp\ipykernel_23412\1534151327.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


Index(['loc_id', 'date', 'animal', 'regis_count', 'removal_count',
       'update_date'],
      dtype='object')

In [17]:
local_mean_score = df.groupby("loc_id").agg(
    avg_rating=("rating", "mean"),
    avg_rating_total=("rating_total", "mean")
).reset_index()

local_mean_score

,loc_id,avg_rating,avg_rating_total
0,KSH001,4.400000,14.000000
1,KSH002,4.754545,104.181818
2,KSH003,4.842105,118.447368
3,KSH004,4.626190,191.190476
4,KSH005,4.822857,67.828571
...,...,...,...
113,TYN008,4.780769,178.115385
114,TYN009,4.350000,110.100000
115,TYN010,4.760000,98.333333
116,TYN011,1.914286,29.714286


In [ ]:
df = df.merge(local_mean_score, how="left", on="loc_id")


,id,name,buss_status,loc_id,address,phone,op_hours,category_id,rating,rating_total,newest_review,longitude,latitude,map_url,website,place_id,update_time,avg_rating,avg_rating_total
0,sal0001,毛樂居寵物美容工作室,OPERATIONAL,NTP029,233臺灣新北市烏來區溫泉街47號,None,45.0,4,0.0,0.0,None,121.551585,24.861427,https://maps.google.com/?cid=11355879125359784900,None,ChIJrcvWcAAHaDQRxPNXA-YvmJ0,2025/10/30 09:43:54,0.00,0.000000
1,sal0002,星光貓舍｜曼赤肯｜英短｜小步舞曲｜英長｜貓咪洗澡美容｜貓咪住宿 ｜新北貓舍,OPERATIONAL,NTP008,239臺灣新北市鶯歌區三鶯路106號,None,0.0,4,5.0,49.0,2025-08-19,121.359142,24.943754,https://maps.google.com/?cid=12376728134590950362,None,ChIJAzTNiZwbaDQR2mvg56n4wqs,2025/10/30 09:43:54,4.08,34.466667
2,sal0003,新同學寵物美容,OPERATIONAL,NTP009,237臺灣新北市三峽區復興路141號,0226716300,48.0,4,5.0,4.0,2025-01-16,121.371062,24.938147,https://maps.google.com/?cid=14774640014360391684,None,ChIJXw9AYwAbaDQRBDA5TNAQCs0,2025/10/30 09:43:54,4.60,86.437500
3,sal0004,妙比寵物美容,OPERATIONAL,NTP009,237臺灣新北市三峽區文化路20號2樓,0226738321,40.0,4,4.8,39.0,2020-11-09,121.373299,24.935908,https://maps.google.com/?cid=13996134824851865085,http://www.facebook.com/mubepet,ChIJzQCMsvMbaDQR_f3E73dCPMI,2025/10/30 09:43:54,4.60,86.437500
4,sal0005,瑪妮寵物,OPERATIONAL,NTP008,239臺灣新北市鶯歌區中正三路278號,0226771300,0.0,4,4.6,7.0,2023-08-18,121.328898,24.936068,https://maps.google.com/?cid=17819326260165638416,None,ChIJ9dHkBD8ZaDQREHn2odL2Svc,2025/10/30 09:43:54,4.08,34.466667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,sal2081,內門內豐台灣犬,OPERATIONAL,KSH035,845臺灣高雄市內門區4之6號,0933393833,0.0,4,4.8,4.0,2025-03-08,120.459281,22.937735,https://maps.google.com/?cid=10985412424350028602,None,ChIJTW7B331pbjQROtOLkGUGdJg,2025/10/30 11:02:40,4.90,64.500000
2081,sal2082,月光山寵物寄宿,OPERATIONAL,KSH034,846臺灣高雄市杉林區桐竹路205號,0928667236,56.0,4,5.0,1.0,2025-09-14,120.532791,22.936386,https://maps.google.com/?cid=6177076079285306252,None,ChIJDS5BIBhDbjQRjANdcPdhuVU,2025/10/30 11:02:40,4.90,3.000000
2082,sal2083,好友犬舍,OPERATIONAL,KSH034,846臺灣高雄市杉林區山仙路93巷14之5號,0928713710,0.0,4,4.8,5.0,2023-10-01,120.534910,22.977118,https://maps.google.com/?cid=15421732636815242056,None,ChIJHxsTVSlDbjQRSLNrFiAABdY,2025/10/30 11:02:40,4.90,3.000000
2083,sal2084,草地人溫泉會館,OPERATIONAL,KSH032,844臺灣高雄市六龜區新開路33之1號,076791992,0.0,4,4.1,1168.0,2025-06-02,120.677556,23.056459,https://maps.google.com/?cid=10060695879360495629,https://165606444270.web.fullinn.tw/,ChIJcVlbAIZZbjQRDcCEMpnFnos,2025/10/30 11:02:40,4.10,1168.000000


In [ ]:
df["op_score"] = 0.6 + ((df["op_hours"] / 168) * 0.4)

,id,name,buss_status,loc_id,address,phone,op_hours,category_id,rating,rating_total,newest_review,longitude,latitude,map_url,website,place_id,update_time,avg_rating,avg_rating_total,op_score
0,sal0001,毛樂居寵物美容工作室,OPERATIONAL,NTP029,233臺灣新北市烏來區溫泉街47號,None,45.0,4,0.0,0.0,None,121.551585,24.861427,https://maps.google.com/?cid=11355879125359784900,None,ChIJrcvWcAAHaDQRxPNXA-YvmJ0,2025/10/30 09:43:54,0.00,0.000000,0.707143
1,sal0002,星光貓舍｜曼赤肯｜英短｜小步舞曲｜英長｜貓咪洗澡美容｜貓咪住宿 ｜新北貓舍,OPERATIONAL,NTP008,239臺灣新北市鶯歌區三鶯路106號,None,0.0,4,5.0,49.0,2025-08-19,121.359142,24.943754,https://maps.google.com/?cid=12376728134590950362,None,ChIJAzTNiZwbaDQR2mvg56n4wqs,2025/10/30 09:43:54,4.08,34.466667,0.600000
2,sal0003,新同學寵物美容,OPERATIONAL,NTP009,237臺灣新北市三峽區復興路141號,0226716300,48.0,4,5.0,4.0,2025-01-16,121.371062,24.938147,https://maps.google.com/?cid=14774640014360391684,None,ChIJXw9AYwAbaDQRBDA5TNAQCs0,2025/10/30 09:43:54,4.60,86.437500,0.714286
3,sal0004,妙比寵物美容,OPERATIONAL,NTP009,237臺灣新北市三峽區文化路20號2樓,0226738321,40.0,4,4.8,39.0,2020-11-09,121.373299,24.935908,https://maps.google.com/?cid=13996134824851865085,http://www.facebook.com/mubepet,ChIJzQCMsvMbaDQR_f3E73dCPMI,2025/10/30 09:43:54,4.60,86.437500,0.695238
4,sal0005,瑪妮寵物,OPERATIONAL,NTP008,239臺灣新北市鶯歌區中正三路278號,0226771300,0.0,4,4.6,7.0,2023-08-18,121.328898,24.936068,https://maps.google.com/?cid=17819326260165638416,None,ChIJ9dHkBD8ZaDQREHn2odL2Svc,2025/10/30 09:43:54,4.08,34.466667,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,sal2081,內門內豐台灣犬,OPERATIONAL,KSH035,845臺灣高雄市內門區4之6號,0933393833,0.0,4,4.8,4.0,2025-03-08,120.459281,22.937735,https://maps.google.com/?cid=10985412424350028602,None,ChIJTW7B331pbjQROtOLkGUGdJg,2025/10/30 11:02:40,4.90,64.500000,0.600000
2081,sal2082,月光山寵物寄宿,OPERATIONAL,KSH034,846臺灣高雄市杉林區桐竹路205號,0928667236,56.0,4,5.0,1.0,2025-09-14,120.532791,22.936386,https://maps.google.com/?cid=6177076079285306252,None,ChIJDS5BIBhDbjQRjANdcPdhuVU,2025/10/30 11:02:40,4.90,3.000000,0.733333
2082,sal2083,好友犬舍,OPERATIONAL,KSH034,846臺灣高雄市杉林區山仙路93巷14之5號,0928713710,0.0,4,4.8,5.0,2023-10-01,120.534910,22.977118,https://maps.google.com/?cid=15421732636815242056,None,ChIJHxsTVSlDbjQRSLNrFiAABdY,2025/10/30 11:02:40,4.90,3.000000,0.600000
2083,sal2084,草地人溫泉會館,OPERATIONAL,KSH032,844臺灣高雄市六龜區新開路33之1號,076791992,0.0,4,4.1,1168.0,2025-06-02,120.677556,23.056459,https://maps.google.com/?cid=10060695879360495629,https://165606444270.web.fullinn.tw/,ChIJcVlbAIZZbjQRDcCEMpnFnos,2025/10/30 11:02:40,4.10,1168.000000,0.600000


In [24]:
loc_data = E_load_from_sql(table_name="location")

df_loc = pd.DataFrame(data=loc_data)

df_loc = df_loc[["loc_id", "population"]]

df_loc

C:\Users\add41\AppData\Local\Temp\ipykernel_21912\1534151327.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,loc_id,population
0,NTP001,550256.0
1,NTP002,383035.0
2,NTP003,404507.0
3,NTP004,211631.0
4,NTP005,421669.0
...,...,...
153,KSH034,10595.0
154,KSH035,12609.0
155,KSH036,1838.0
156,KSH037,4137.0


In [26]:
df_loc["m"] = 30 * ((df_loc["population"]/10000)**0.33)
df_loc

,loc_id,population,m
0,NTP001,550256.0,112.592026
1,NTP002,383035.0,99.905626
2,NTP003,404507.0,101.720116
3,NTP004,211631.0,82.141460
4,NTP005,421669.0,103.124512
...,...,...,...
153,KSH034,10595.0,30.577683
154,KSH035,12609.0,32.385147
155,KSH036,1838.0,17.153600
156,KSH037,4137.0,22.419580


In [27]:
df = df.merge(df_loc, how="left", on="loc_id")
df

,id,name,buss_status,loc_id,address,phone,op_hours,category_id,rating,rating_total,...,map_url,website,place_id,update_time,avg_rating,avg_rating_total,op_score,store_score,population,m
0,sal0001,毛樂居寵物美容工作室,OPERATIONAL,NTP029,233臺灣新北市烏來區溫泉街47號,None,45.0,4,0.0,0.0,...,https://maps.google.com/?cid=11355879125359784900,None,ChIJrcvWcAAHaDQRxPNXA-YvmJ0,2025/10/30 09:43:54,0.00,0.000000,0.707143,0.000000,6164.0,25.572661
1,sal0002,星光貓舍｜曼赤肯｜英短｜小步舞曲｜英長｜貓咪洗澡美容｜貓咪住宿 ｜新北貓舍,OPERATIONAL,NTP008,239臺灣新北市鶯歌區三鶯路106號,None,0.0,4,5.0,49.0,...,https://maps.google.com/?cid=12376728134590950362,None,ChIJAzTNiZwbaDQR2mvg56n4wqs,2025/10/30 09:43:54,4.08,34.466667,0.600000,46.274880,89581.0,61.851823
2,sal0003,新同學寵物美容,OPERATIONAL,NTP009,237臺灣新北市三峽區復興路141號,0226716300,48.0,4,5.0,4.0,...,https://maps.google.com/?cid=14774640014360391684,None,ChIJXw9AYwAbaDQRBDA5TNAQCs0,2025/10/30 09:43:54,4.60,86.437500,0.714286,59.658929,115077.0,67.181156
3,sal0004,妙比寵物美容,OPERATIONAL,NTP009,237臺灣新北市三峽區文化路20號2樓,0226738321,40.0,4,4.8,39.0,...,https://maps.google.com/?cid=13996134824851865085,http://www.facebook.com/mubepet,ChIJzQCMsvMbaDQR_f3E73dCPMI,2025/10/30 09:43:54,4.60,86.437500,0.695238,81.316786,115077.0,67.181156
4,sal0005,瑪妮寵物,OPERATIONAL,NTP008,239臺灣新北市鶯歌區中正三路278號,0226771300,0.0,4,4.6,7.0,...,https://maps.google.com/?cid=17819326260165638416,None,ChIJ9dHkBD8ZaDQREHn2odL2Svc,2025/10/30 09:43:54,4.08,34.466667,0.600000,20.738880,89581.0,61.851823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,sal2081,內門內豐台灣犬,OPERATIONAL,KSH035,845臺灣高雄市內門區4之6號,0933393833,0.0,4,4.8,4.0,...,https://maps.google.com/?cid=10985412424350028602,None,ChIJTW7B331pbjQROtOLkGUGdJg,2025/10/30 11:02:40,4.90,64.500000,0.600000,40.230000,12609.0,32.385147
2081,sal2082,月光山寵物寄宿,OPERATIONAL,KSH034,846臺灣高雄市杉林區桐竹路205號,0928667236,56.0,4,5.0,1.0,...,https://maps.google.com/?cid=6177076079285306252,None,ChIJDS5BIBhDbjQRjANdcPdhuVU,2025/10/30 11:02:40,4.90,3.000000,0.733333,2.889333,10595.0,30.577683
2082,sal2083,好友犬舍,OPERATIONAL,KSH034,846臺灣高雄市杉林區山仙路93巷14之5號,0928713710,0.0,4,4.8,5.0,...,https://maps.google.com/?cid=15421732636815242056,None,ChIJHxsTVSlDbjQRSLNrFiAABdY,2025/10/30 11:02:40,4.90,3.000000,0.600000,4.644000,10595.0,30.577683
2083,sal2084,草地人溫泉會館,OPERATIONAL,KSH032,844臺灣高雄市六龜區新開路33之1號,076791992,0.0,4,4.1,1168.0,...,https://maps.google.com/?cid=10060695879360495629,https://165606444270.web.fullinn.tw/,ChIJcVlbAIZZbjQRDcCEMpnFnos,2025/10/30 11:02:40,4.10,1168.000000,0.600000,1149.312000,11329.0,31.261116


In [28]:
df["store_score"] = (((df["rating"]/5) * (df["rating_total"]/(df["rating_total"]+df["m"]))
                      ) + ((df["avg_rating"]/5) * (df["m"]/(df["avg_rating_total"]+df["m"])))) * df["op_score"]

In [31]:
df.groupby("loc_id")["store_score"].sum()

loc_id
KSH001     0.611810
KSH002     6.707988
KSH003    22.520977
KSH004    19.305007
KSH005    22.637417
            ...    
TYN008    13.901938
TYN009    10.059359
TYN010    16.701194
TYN011     2.192633
TYN012     5.649738
Name: store_score, Length: 118, dtype: float64

In [10]:
from pathlib import Path
from mods import GCS_mod as gcs

data_type = "pet_regis"

data_loc = E_load_from_sql(table_name=data_type)
df_loc = pd.DataFrame(data_loc)

folder = Path(r"C:\Users\add41\Documents\Data_Engineer\Project\example_data")
file_name = f"{data_type}.csv"
path = folder / file_name

df_loc.to_csv(path, index=False, encoding="utf-8-sig")

# gcs_setting = {
#     "bucket_name":"tjr103-1-project-bucket",
#     "destination":f"data/complete/store/type={data_type}/store.csv",
#     "source_file_name":path
# }

# gcs.L_upload_to_gcs(gcs_setting=gcs_setting)

C:\Users\add41\AppData\Local\Temp\ipykernel_21048\1534151327.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [6]:
def L_upload_data_to_db(df: pd.DataFrame, sql: str):
    conn = create_pymysql_connect()
    cursor = conn.cursor()

    data = list(df.itertuples(index=False, name=None))

    try:
        cursor.executemany(sql, data)
        conn.commit()
        print("資料寫入資料庫成功！")
    except Exception as e:
        print(f"資料寫入資料褲時發生錯誤：{e}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

In [7]:
df = pd.DataFrame({
    "ani_code":[2, 3, 4],
    "animal": ["bird", "bull", "duck"]
})

sql = "INSERT INTO animal (ani_code, animal)" \
        "VALUES(%s, %s)"

L_upload_data_to_db(df=df, sql=sql)

資料寫入資料庫成功！
